In [1]:
import cobra
import reframed
import FLUX_DISTANCE
import cplex
import sys
from cobra import Model, Reaction, Metabolite

In [2]:
print(sys.stdout)

In [3]:
Sc = cobra.io.read_sbml_model('C:/Users/pjouh/OneDrive/Documents/yeast-GEM-8.6.0/yeast-GEM-8.6.0/model/yeast-GEM.xml')

In [4]:
for rxn in Sc.reactions:
        print(Sc.reactions.get_by_id(rxn.id))

r_0001: s_0025 + 2.0 s_0709 --> 2.0 s_0710 + s_1399
r_0002: s_0027 + 2.0 s_0709 --> 2.0 s_0710 + s_1401
r_0003: s_0035 + s_1198 <=> s_0020 + s_0794 + s_1203
r_0004: s_0063 + 2.0 s_0709 --> 2.0 s_0710 + s_1399
r_0005: s_1543 --> s_0001 + s_0794 + s_1538
r_0006: s_1543 --> s_0004 + s_0794 + s_1538
r_0007: s_0077 --> s_0312
r_0012: s_0119 + 2.0 s_0807 + s_1200 --> s_0799 + s_0993 + s_1205
r_0013: s_0311 + 3.0 s_0803 --> s_0294 + s_0722 + 6.0 s_0794 + s_1322
r_0014: s_0142 + s_0794 + s_0803 --> s_0313 + s_0419
r_0015: s_0141 + s_0794 + s_1212 --> s_0142 + s_1207
r_0016: s_0179 + s_0799 + s_1401 --> s_0039 + s_0460
r_0017: s_0148 + s_0437 --> s_0006 + s_0424 + s_0799
r_0018: s_0176 + s_0991 <=> s_0180 + s_0953
r_0019: s_0149 + s_0794 + s_1212 --> s_0030 + s_1207
r_0020: s_0552 + s_0807 + s_1361 --> s_0350 + s_1326
r_0021: s_0155 + s_1419 --> s_0156 + s_0799 + s_1415
r_0022: s_0157 + s_1278 --> s_0155 + s_0807
r_0023: s_0165 + s_0803 <=> s_0162
r_0024: s_0232 + s_0373 + s_0803 --> s_0162 + s

r_3765: s_3109 <=> s_3128
r_3766: s_3110 <=> s_3131
r_3767: s_3111 <=> s_3133
r_3768: s_3112 <=> s_3135
r_3769: s_3113 <=> s_3137
r_3770: s_3114 <=> s_3139
r_3771: s_3115 <=> s_3141
r_3772: s_3116 <=> s_3143
r_3773: s_3130 <=> s_3181
r_3774: s_3132 <=> s_3185
r_3775: s_3134 <=> s_3187
r_3776: s_3136 <=> s_3189
r_3777: s_3138 <=> s_3191
r_3778: s_3140 <=> s_3193
r_3779: s_3142 <=> s_3195
r_3780: s_3144 <=> s_3197
r_3781: s_3207 <=> s_3296
r_3782: s_3208 <=> s_3298
r_3783: s_3209 <=> s_3300
r_3784: s_3210 <=> s_3302
r_3785: s_3211 <=> s_3304
r_3786: s_3212 <=> s_3305
r_3787: s_3213 <=> s_3306
r_3788: s_3214 <=> s_3307
r_3789: s_2967 <=> s_2996
r_3790: s_2969 <=> s_3000
r_3791: s_2971 <=> s_3004
r_3792: s_2973 <=> s_3008
r_3793: s_2968 <=> s_2998
r_3794: s_2970 <=> s_3002
r_3795: s_2972 <=> s_3006
r_3796: s_2974 <=> s_3010
r_3797: s_2954 <=> s_2993
r_3798: s_2956 <=> s_2999
r_3799: s_2958 <=> s_3003
r_3800: s_2960 <=> s_3007
r_3801: s_2955 <=> s_2997
r_3802: s_2957 <=> s_3001
r_3803: s_29

In [5]:
original_stdout = sys.stdout
with open('yeast-GEM_86_rxns.txt','w') as outfile:
    sys.stdout = outfile
    for rxn in Sc.reactions:
        print(str(rxn.id) +"\t"+ rxn.build_reaction_string(use_metabolite_names=True))
        #print(rxn.build_reaction_string(use_metabolite_names=True))
    sys.stdout = original_stdout
outfile.close

<function TextIOWrapper.close()>

In [6]:
with open('yeast-GEM_86_mets.txt','w') as f:
    sys.stdout = f
    for met in Sc.metabolites:
        id = Sc.metabolites.get_by_id(met.id)
        nimi = Sc.metabolites.get_by_id(met.id).name
        print(nimi+'\t'+str(id))
    sys.stdout = original_stdout
f.close

<function TextIOWrapper.close()>

In [7]:
Sc_objective = Sc.objective

In [8]:
print(Sc_objective)

Maximize
1.0*r_2111 - 1.0*r_2111_reverse_58b69


In [9]:
dhs=Sc.metabolites.get_by_id('s_0211')
h2o=Sc.metabolites.get_by_id('s_0803')
h=Sc.metabolites.get_by_id('s_0794')
o2=Sc.metabolites.get_by_id('s_1275')
co2=Sc.metabolites.get_by_id('s_0456')

In [10]:
#introduce heterologous muconic acid synthesis pathway in yeast
reaction = Reaction('PaAroZ')
reaction.name = '3-dehydroshikimate dehydratase'
reaction.subsystem = 'aromatic compound metabolism'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [11]:
PCA_c = Metabolite(
    'protocatechuate_c',
    formula='C7H6O4',
    name='protocatechuate',
    compartment='c')
catechol_c = Metabolite(
    'catechol_c',
    formula='C6H6O2',
    name='catechol',
    compartment='c')
muconic_acid_c = Metabolite(
    'muconic_acid_c',
    formula='C6H4O4',
    name='muconic_acid',
    compartment='c')
muconic_acid_e = Metabolite(
    'muconic_acid_e',
    formula='C6H4O4',
    name='muconic_acid',
    compartment='e')

In [12]:
reaction.add_metabolites({
    dhs: -1.0,
    PCA_c: 1.0,
    h2o: 1.0
})

reaction.reaction 

's_0211 --> protocatechuate_c + s_0803'

In [13]:
Sc.add_reactions([reaction])

In [14]:
reaction = Reaction('KpAroY')
reaction.name = 'Protocatechuate decarboxylase'
reaction.subsystem = 'aromatic compound metabolism'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [15]:
reaction.add_metabolites({
    PCA_c: -1.0,
    h: -1.0,
    catechol_c: 1.0,
    co2: 1.0
})

reaction.reaction 

'protocatechuate_c + s_0794 --> catechol_c + s_0456'

In [16]:
Sc.add_reactions([reaction])

In [17]:
reaction = Reaction('CaHqd2')
reaction.name = 'Catechol 1,2-dioxygenase'
reaction.subsystem = 'aromatic compound metabolism'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [18]:
reaction.add_metabolites({
    catechol_c: -1.0,
    o2: -1.0,
    muconic_acid_c: 1.0
})

reaction.reaction 

'catechol_c + s_1275 --> muconic_acid_c'

In [19]:
Sc.add_reactions([reaction])

In [20]:
reaction = Reaction('Muconic_acid_transport')
reaction.name = 'Muconic_acid_transport'
reaction.subsystem = 'transport'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [21]:
reaction.add_metabolites({
    muconic_acid_c: -1.0,
    muconic_acid_e: 1.0
})

reaction.reaction 

'muconic_acid_c --> muconic_acid_e'

In [22]:
Sc.add_reactions([reaction])

In [23]:
reaction = Reaction('EX_muconic_acid')
reaction.name = 'Muconic_acid_exchange'
reaction.subsystem = 'exchange'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default

In [24]:
reaction.add_metabolites({
    muconic_acid_e: -1.0
})

reaction.reaction 

'muconic_acid_e --> '

In [25]:
Sc.add_reactions([reaction])

In [26]:
solution=Sc.optimize()

In [27]:
solution

,fluxes,reduced_costs
r_0001,0.000000,-0.017722
r_0002,0.000000,-0.017722
r_0003,0.000000,0.000000
r_0004,0.000000,0.000000
r_0005,0.061324,0.000000
...,...,...
PaAroZ,0.000000,0.000000
KpAroY,0.000000,0.000000
CaHqd2,0.000000,0.000000
Muconic_acid_transport,0.000000,0.000000


In [28]:
#let's run flux distance to all MA pathway fluxes
flux_distance_to_source = FLUX_DISTANCE.get_flux_distances(Sc,'PaAroZ','PaAroZ')

316
(2753, 4074)
(2437, 4074)
(2437, 3799)


In [29]:
#let's run flux distance to all MA pathway fluxes
flux_distance_to_source = FLUX_DISTANCE.get_flux_distances(Sc,'KpAroY','KpAroY')

316
(2753, 4074)
(2437, 4074)
(2437, 3799)


In [30]:
#let's run flux distance to all MA pathway fluxes
flux_distance_to_source = FLUX_DISTANCE.get_flux_distances(Sc,'CaHqd2','CaHqd2')

316
(2753, 4074)
(2437, 4074)
(2437, 3799)


In [31]:
print(flux_distance_to_source)

{'r_0001': 7.0, 'r_0002': 8.0, 'r_0003': 8.0, 'r_0004': 7.0, 'r_0005': 9.0, 'r_0006': 9.0, 'r_0007': 9.0, 'r_0012': 9.0, 'r_0013': 9.0, 'r_0014': 11.0, 'r_0015': 12.0, 'r_0016': 8.0, 'r_0017': 10.0, 'r_0018': 8.0, 'r_0019': 10.0, 'r_0020': 6.0, 'r_0021': 10.0, 'r_0022': 11.0, 'r_0023': 10.0, 'r_0024': 9.0, 'r_0025': 8.0, 'r_0026': 8.0, 'r_0027': 9.0, 'r_0028': 8.0, 'r_0029': 9.0, 'r_0030': 9.0, 'r_0032': 9.0, 'r_0033': 10000.0, 'r_0034': 10000.0, 'r_0035': 10000.0, 'r_0036': 10000.0, 'r_0037': 10000.0, 'r_0038': 8.0, 'r_0039': 3.0, 'r_0040': 4.0, 'r_0041': 10.0, 'r_0042': 5.0, 'r_0043': 11.0, 'r_0044': 11.0, 'r_0045': 8.0, 'r_0057': 10.0, 'r_0058': 9.0, 'r_0059': 9.0, 'r_0060': 10.0, 'r_0061': 10.0, 'r_0062': 9.0, 'r_0063': 9.0, 'r_0064': 9.0, 'r_0065': 5.0, 'r_0066': 7.0, 'r_0067': 7.0, 'r_0068': 8.0, 'r_0069': 10000.0, 'r_0070': 9.0, 'r_0072': 9.0, 'r_0073': 17.0, 'r_0074': 18.0, 'r_0075': 10.0, 'r_0076': 11.0, 'r_0077': 10.0, 'r_0078': 10.0, 'r_0079': 8.0, 'r_0080': 9.0, 'r_0081': 9

In [32]:
from cobra.flux_analysis import flux_variability_analysis

In [33]:
fva_solution = flux_variability_analysis(Sc,fraction_of_optimum=0.97)

In [34]:
Sc.objective="EX_muconic_acid"

In [35]:
solution_MA=Sc.optimize()

In [36]:
solution_MA

,fluxes,reduced_costs
r_0001,0.000000,-0.22229
r_0002,0.000000,-0.22229
r_0003,0.000000,0.00000
r_0004,0.000000,0.00000
r_0005,0.000000,0.00000
...,...,...
PaAroZ,0.848594,0.00000
KpAroY,0.848594,0.00000
CaHqd2,0.848594,0.00000
Muconic_acid_transport,0.848594,0.00000


In [37]:
fva_solution_MA = flux_variability_analysis(Sc,fraction_of_optimum=0.97)

In [38]:
cobra.io.write_sbml_model(Sc,'C:/Users/pjouh/OneDrive/Documents/Sc_MA.xml')

In [39]:
from reframed import load_cbmodel
Sc_MA = load_cbmodel('C:/Users/pjouh/OneDrive/Documents/Sc_MA.xml')

In [40]:
reframed.cobra.simulation.FBA(Sc_MA)

Objective: 0.8485942423593479
Status: Optimal

In [41]:
Sc_MA_pFBA = reframed.cobra.simulation.pFBA(Sc_MA)

In [42]:
with open('Sc_MA_pFBA.csv', 'w') as f:
    for key in Sc_MA_pFBA.values.keys():
        f.write("%s,%s\n"%(key, Sc_MA_pFBA.values[key]))

In [43]:
Sc_MA.set_objective({'R_r_2111': 1,'R_EX_muconic_acid': 0})

In [44]:
Sc_MA_pFBA_growth = reframed.cobra.simulation.pFBA(Sc_MA)

In [45]:
print(Sc_MA_pFBA)

Objective: 47.51909015816689
Status: Optimal



In [46]:
print(Sc_MA_pFBA_growth)

Objective: 99.98598176631542
Status: Optimal



In [47]:
optgrowth = reframed.cobra.simulation.FBA(Sc_MA)

In [48]:
optvalue = optgrowth.values['R_r_2111']

In [49]:
Sc_MA.get_objective()

{'R_r_2111': 1}

In [50]:
Sc_MA

R_r_0001: M_s_0025 + 2.0 M_s_0709 --> 2.0 M_s_0710 + M_s_1399
R_r_0002: M_s_0027 + 2.0 M_s_0709 --> 2.0 M_s_0710 + M_s_1401
R_r_0003: M_s_0035 + M_s_1198 <-> M_s_0020 + M_s_0794 + M_s_1203
R_r_0004: M_s_0063 + 2.0 M_s_0709 --> 2.0 M_s_0710 + M_s_1399
R_r_0005: M_s_1543 --> M_s_0001 + M_s_0794 + M_s_1538
R_r_0006: M_s_1543 --> M_s_0004 + M_s_0794 + M_s_1538
R_r_0007: M_s_0077 --> M_s_0312
R_r_0012: M_s_0119 + 2.0 M_s_0807 + M_s_1200 --> M_s_0799 + M_s_0993 + M_s_1205
R_r_0013: M_s_0311 + 3.0 M_s_0803 --> M_s_0294 + M_s_0722 + 6.0 M_s_0794 + M_s_1322
R_r_0014: M_s_0142 + M_s_0794 + M_s_0803 --> M_s_0313 + M_s_0419
R_r_0015: M_s_0141 + M_s_0794 + M_s_1212 --> M_s_0142 + M_s_1207
R_r_0016: M_s_0179 + M_s_0799 + M_s_1401 --> M_s_0039 + M_s_0460
R_r_0017: M_s_0148 + M_s_0437 --> M_s_0006 + M_s_0424 + M_s_0799
R_r_0018: M_s_0176 + M_s_0991 <-> M_s_0180 + M_s_0953
R_r_0019: M_s_0149 + M_s_0794 + M_s_1212 --> M_s_0030 + M_s_1207
R_r_0020: M_s_0552 + M_s_0807 + M_s_1361 --> M_s_0350 + M_s_1326
R

In [51]:
print(Sc_MA_pFBA)

Objective: 47.51909015816689
Status: Optimal



In [52]:
constraints = {'R_r_2111': (0.9*optvalue,1000)}

In [53]:
Sc_MA_lMoMA=reframed.cobra.simulation.lMOMA(Sc_MA, reference=Sc_MA_pFBA.values, constraints=constraints, reactions=None, solver=None)

In [54]:
Sc_MA_lMoMA.values

{'R_r_0001': 0.0,
 'R_r_0002': 0.0,
 'R_r_0003': 0.0,
 'R_r_0004': 0.0,
 'R_r_0005': 0.055191944315120124,
 'R_r_0006': 0.01844057276866301,
 'R_r_0007': 0.005615470558314044,
 'R_r_0012': 0.0,
 'R_r_0013': 1.400969954773641e-05,
 'R_r_0014': 7.37352562739179e-05,
 'R_r_0015': 7.37352562739179e-05,
 'R_r_0016': 0.016321285269641748,
 'R_r_0017': 0.0,
 'R_r_0018': 0.024240537957700294,
 'R_r_0019': 1.400969954773641e-05,
 'R_r_0020': 0.030995403906185263,
 'R_r_0021': 0.0,
 'R_r_0022': 0.0,
 'R_r_0023': -0.025104458543905337,
 'R_r_0024': 0.0,
 'R_r_0025': 0.025104458543905337,
 'R_r_0026': 1.400969954773641e-05,
 'R_r_0027': 0.024240537957700294,
 'R_r_0028': 0.0,
 'R_r_0029': 0.025104458543905337,
 'R_r_0030': 0.0,
 'R_r_0032': 0.0048672879858626256,
 'R_r_0033': 0.0,
 'R_r_0034': 0.0,
 'R_r_0035': 0.0,
 'R_r_0036': 0.0,
 'R_r_0037': 0.0,
 'R_r_0038': 0.0001474705125478358,
 'R_r_0039': 0.030995403906185263,
 'R_r_0040': 0.030995403906185263,
 'R_r_0041': 0.0,
 'R_r_0042': 0.0,
 'R_r_

In [55]:
with open('Sc_MA_lMoMA.csv', 'w') as f:
    for key in Sc_MA_lMoMA.values.keys():
        f.write("%s,%s\n"%(key, Sc_MA_lMoMA.values[key]))